### Creating a Delta Table with the DeltaTableBuilder API

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

from delta import *

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]
warehouse_dir = f"{work_dir}/spark-warehouse"

# Create spark session with hive enabled
builder = (
    SparkSession
    .builder
    .appName("pyspark-notebook")
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.warehouse.dir", warehouse_dir)
    .enableHiveSupport()
)

In [2]:
# 델타 레이크 생성시에 반드시 `configure_spark_with_delta_pip` 구성을 통해 실행되어야 정상적인 델타 의존성이 로딩됩니다
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

In [10]:
# DeltaTable API 수준에서 Drop 하는 API 제공되지 않음
spark.sql("""
DROP TABLE IF EXISTS taxidb.greenTaxis
""")

""


In [11]:
deltaTable = (
    DeltaTable.createIfNotExists(spark)
    .tableName("taxidb.greenTaxis")
    .addColumn("VendorId", "INT", comment = "Ride Vendor")
    .addColumn("lpep_pickup_datetime", "STRING")
    .addColumn("lpep_dropoff_datetime", "STRING")
    .addColumn("store_and_fwd_flag", "STRING")
    .addColumn("RatecodeID", "INT", comment = "Ref to RateCard")
    .addColumn("PULocationID", "INT")
    .addColumn("DOLocationID", "INT")
    .addColumn("passenger_count", "INT")
    .addColumn("trip_distance", "DOUBLE")
    .addColumn("fare_amount", "DOUBLE")
    .addColumn("extra", "DOUBLE")
    .addColumn("mta_tax", "DOUBLE")
    .addColumn("tip_amount", "DOUBLE")
    .addColumn("tolls_amount", "DOUBLE")
    .addColumn("ehail_fee", "STRING")
    .addColumn("improvement_surcharge", "DOUBLE")
    .addColumn("total_amount", "DOUBLE")
    .addColumn("payment_type", "INT")
    .addColumn("trip_type", "INT")
    .addColumn("congestion_surcharge", "DOUBLE")
)
deltaTable.execute()

In [12]:
print(work_dir)
!ls {work_dir}/spark-warehouse/taxidb.db/greentaxis/_delta_log

/home/jovyan/work
00000000000000000000.json


In [13]:
csvGreenTaxi = spark.read.format("csv").option("inferSchema", True).option("header", True).load(f"{work_dir}/data/greenTaxis.csv")
csvGreenTaxi.printSchema()
csvGreenTaxi.show(10, truncate=False)

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

+--------+--------------------+---------------------+------------------+----------+-------

In [14]:
csvGreenTaxi.write.format("delta").mode("overwrite").saveAsTable("taxidb.greenTaxis")

In [15]:
!ls {work_dir}/spark-warehouse/taxidb.db/greentaxis/_delta_log
!cat {work_dir}/spark-warehouse/taxidb.db/greentaxis/_delta_log/00000000000000000001.json

00000000000000000000.json  00000000000000000001.json
{"add":{"path":"part-00000-565658ad-e550-4167-b9c9-5a360ee8049c-c000.snappy.parquet","partitionValues":{},"size":930701,"modificationTime":1723631962161,"dataChange":true,"stats":"{\"numRecords\":46087,\"minValues\":{\"VendorId\":1,\"lpep_pickup_datetime\":\"2009-01-01 00:10:36\",\"lpep_dropoff_datetime\":\"2009-01-01 00:20:29\",\"store_and_fwd_flag\":\"N\",\"RatecodeID\":1,\"PULocationID\":2,\"DOLocationID\":1,\"passenger_count\":0,\"trip_distance\":0.0,\"fare_amount\":-52.0,\"extra\":-1.0,\"mta_tax\":-0.5,\"tip_amount\":-0.76,\"tolls_amount\":0.0,\"improvement_surcharge\":-0.3,\"total_amount\":-52.8,\"payment_type\":1,\"trip_type\":1,\"congestion_surcharge\":0.0},\"maxValues\":{\"VendorId\":2,\"lpep_pickup_datetime\":\"2019-12-05 17:46:27\",\"lpep_dropoff_datetime\":\"2019-12-06 06:12:54\",\"store_and_fwd_flag\":\"Y\",\"RatecodeID\":5,\"PULocationID\":265,\"DOLocationID\":265,\"passenger_count\":8,\"trip_distance\":74.6,\"fare_amou

In [30]:
# 테이블 드랍의 경우 모든 메타 정보와 데이터 파일은 유지되며, 데이터베이스에 테이블 정보만 삭제됨
spark.sql("DROP TABLE IF EXISTS taxidb.rateCard")

""


In [31]:
!ls {work_dir}/data/rateCard.delta/_delta_log
!cat {work_dir}/data/rateCard.delta/_delta_log/00000000000000000001.json

00000000000000000000.json  00000000000000000001.json
{"add":{"path":"part-00000-879dbb89-ee5a-47d4-ab0d-0e6445663607-c000.snappy.parquet","partitionValues":{},"size":858,"modificationTime":1723625669220,"dataChange":true,"stats":"{\"numRecords\":6,\"minValues\":{\"rateCodeId\":1,\"rateCodeDesc\":\"Group ride\"},\"maxValues\":{\"rateCodeId\":6,\"rateCodeDesc\":\"Standard Rate\"},\"nullCount\":{\"rateCodeId\":0,\"rateCodeDesc\":0}}"}}
{"commitInfo":{"timestamp":1723625668130,"operation":"CREATE OR REPLACE TABLE AS SELECT","operationParameters":{"isManaged":"false","description":null,"partitionBy":"[]","properties":"{}"},"readVersion":0,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"1","numOutputRows":"6","numOutputBytes":"858"},"engineInfo":"Apache-Spark/3.2.1 Delta-Lake/2.0.0","txnId":"5e49e955-4dca-4948-89a1-32cf38bb834f"}}


In [16]:
spark.sql("use taxidb")
spark.sql("show tables")

namespace,tableName,isTemporary
taxidb,greentaxis,false
taxidb,ratecard,false


In [18]:
spark.sql("""
select * from taxidb.greentaxis
""").show(5, truncate=False)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorId|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|2       |2019-12-19 17:12:23 |2019-12-19 17:19:32  |N                 |1         |181         |25          |1              |1.28         |7.0        |1.0  |0.5   

In [19]:
spark.sql("select count(1) from taxidb.greentaxis")

count(1)
450627


In [23]:
spark.sql("""
SELECT VendorID, AVG(fare_amount) AS AverageFare
FROM taxidb.greentaxis
GROUP BY VendorID
HAVING AVG(fare_amount) > 10
ORDER BY 2 DESC
LIMIT 5
""")

VendorID,AverageFare
null,28.830541636814186
1,13.217967034800125
2,12.054380250700097


In [24]:
# Reading a Table with PySpark
df = spark.read.format("delta").table("taxidb.greentaxis")
print(f"Number of records: {df.count():,}")

Number of records: 450,627


In [25]:
res = (
    df.groupBy("VendorID")
    .agg(avg("fare_amount").alias("AverageFare"))
    .filter(col("AverageFare") > 10)
    .sort(col("AverageFare").desc())
)
res.take(5)

[Row(VendorID=None, AverageFare=28.830541636814186),
 Row(VendorID=1, AverageFare=13.217967034800125),
 Row(VendorID=2, AverageFare=12.054380250700097)]

In [27]:
spark.sql(f"""
CREATE TABLE taxidb.YellowTaxis (
    RideId INT,
    VendorId INT,
    PickupTime TIMESTAMP,
    DropTime TIMESTAMP,
    PickupLocationId INT,
    DropLocationId INT,
    CabNumber STRING,
    DriverLicenseNumber STRING,
    PassengerCount INT,
    TripDistance DOUBLE,
    RatecodeId INT,
    PaymentType INT,
    TotalAmount DOUBLE,
    FareAmount DOUBLE,
    Extra DOUBLE,
    MtaTax DOUBLE,
    TipAmount DOUBLE,
    TollsAmount DOUBLE,
    ImprovementSurcharge DOUBLE
) USING DELTA
LOCATION "{work_dir}/data/yellowTaxis.delta"
""")

""


In [28]:
spark.sql("""
INSERT INTO taxidb.yellowtaxis (RideId, VendorId, PickupTime, DropTime, PickupLocationId,
    DropLocationId, CabNumber, DriverLicenseNumber, PassengerCount, TripDistance,
    RatecodeId, PaymentType, TotalAmount, FareAmount, Extra, 
    MtaTax, TipAmount, TollsAmount, ImprovementSurcharge)
VALUES (9999995, 1, '2019-11-01T00:00:00.000Z', '2019-11-01T00:02:23.573Z', 65, 
    71, 'TAC304', '453987', 2, 4.5, 
    1, 1, 20.34, 15.0, 0.5, 
    0.4, 2.0, 2.0, 1.1)
""")
spark.sql("SELECT count(RideId) AS count FROM taxidb.YellowTaxis WHERE RideId = 9999995")

count
1


### Using the OverWrite Mode When Writing to a Delta Table

In [ ]:
# Appending a DataFrame to a Table @ https://drive.google.com/file/d/1--SfboYb-KyEug4U89m2pnjq0MyUWbvJ/view?usp=sharing
yellowTaxi = spark.read.format("delta").table("taxidb.YellowTaxis")
yellowTaxiSchema = yellowTaxi.schema
print(yellowTaxiSchema)

df_for_append = (
    spark.read
    .option("header", True)
    .parquet("/mnt/datalake/book/data files/YellowTaxis_append.csv")


In [29]:
df = spark.read.parquet(f"{work_dir}/data/yellowTaxis.parquet")
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [30]:
df.show(3, truncate=False)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|1       |2021-01-01 09:30:10 |2021-01-01 09:36:12  |1.0            |2.1          |1.0       |N                 |142         |43          |2           |8.0        |3.0  |0.5    |0.0      

In [37]:
filtered = df.where(expr("VendorID = 1 and PULocationID = 142 and DOLocationID = 43"))

In [40]:
filtered.show(1, truncate=False)
filtered.groupBy("payment_type").count()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|1       |2021-01-01 09:30:10 |2021-01-01 09:36:12  |1.0            |2.1          |1.0       |N                 |142         |43          |2           |8.0        |3.0  |0.5    |0.0      

payment_type,count
4,3
0,19
3,8
2,143
1,604


In [41]:
filtered.where(expr("payment_type = 3")).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-01-11 01:25:02|  2021-01-11 01:26:18|            1.0|          0.4|       1.0|                 N|         142|          43|           3|        3.5|  3.5|    0.5|       0.

In [42]:
withColumn = filtered.where(expr("payment_type = 3")).withColumn("payment_type", lit(30))
withColumn.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: integer (nullable = false)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [43]:
withColumn.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-01-11 01:25:02|  2021-01-11 01:26:18|            1.0|          0.4|       1.0|                 N|         142|          43|          30|        3.5|  3.5|    0.5|       0.

In [ ]:
# Inserting Data with the SQL COPY INTO Command
spark.sql("""
COPY INTO taxidb.yellowtaxis
FROM (
    SELECT RideId::Int
    , VendorId::Int
    , PickupTime::Timestamp
    , DropTime::Timestamp
    , PickupLocationId::Int
    , DropLocationId::Int
    , CabNumber::String
    , DriverLicenseNumber::String
    , PassengerCount::Int
    , TripDistance::Double
    , RateCodeId::Int
    , PaymentType::Int
    , TotalAmount::Double
    , FareAmount::Double
    , Extra::Double
    , MtaTax::Double
    , TipAmount::Double
    , TollsAmount::Double
    , ImprovementSurcharge::Double
) FROM '/mnt/datalake/book/DataFiles/YellowTaxisLargeAppend.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ("header" = "true")
""")

spark.sql("SELECT COUNT(*) FROM taxidb.YellowTaxis")

### Partitions
> These use cases lend themselves well to partitioning. Partitioning your data to align
with your query patterns can dramatically speed up query performance, especially
when combined with other performance optimizations, such as Z-ordering.7 A Delta
table partition is composed of a folder with a subset of data rows that share the same
value for one or more column(s).

In [ ]:
# 파티션 구성된 델타 테이블을 사전에 생성해 두고
spark.sql(f"""
CREATE TABLE taxidb.YellowTaxisPartitioned
    RideId INT,
    VendorId INT,
    PickupTime TIMESTAMP,
    DropTime TIMESTAMP,
    PickupLocationId INT,
    DropLocationId INT,
    CabNumber STRING,
    DriverLicenseNumber STRING,
    PassengerCount INT,
    TripDistance DOUBLE,
    RatecodeId INT,
    PaymentType INT,
    TotalAmount DOUBLE,
    FareAmount DOUBLE,
    Extra DOUBLE,
    MtaTax DOUBLE,
    TipAmount DOUBLE,
    TollsAmount DOUBLE,
    ImprovementSurcharge DOUBLE
) USING DELTA
PARTITIONED BY(VendorId)
LOCATION "/mnt/datalake/book/chapter03/YellowTaxisDeltaPartitioned"
""")

In [ ]:
# 기존에 생성되어 있는 관리 테이블을 읽어서
df = spark.read.format("delta").table("taxidb.YellowTaxis")
# 델타 테이블에 덮어쓰기를 한다
df.write.format("delta").mode("overwrite").save("/mnt/datalake/book/chapter03/YellowTaxisDeltaPartitioned")

In [ ]:
spark.sql("SELECT DISTINCT(VendorId) FROM taxidb.YellowTaxisPartitioned")

In [ ]:
# Partitioning by multiple columns -- Partition by VendorId AND rateCodeId
spark.sql(f"""
CREATE TABLE taxidb.YellowTaxisPartitioned
    RideId INT,
    VendorId INT,
    PickupTime TIMESTAMP,
    DropTime TIMESTAMP,
    PickupLocationId INT,
    DropLocationId INT,
    CabNumber STRING,
    DriverLicenseNumber STRING,
    PassengerCount INT,
    TripDistance DOUBLE,
    RatecodeId INT,
    PaymentType INT,
    TotalAmount DOUBLE,
    FareAmount DOUBLE,
    Extra DOUBLE,
    MtaTax DOUBLE,
    TipAmount DOUBLE,
    TollsAmount DOUBLE,
    ImprovementSurcharge DOUBLE
) USING DELTA
PARTITIONED BY(VendorId, RatecodeId)
LOCATION "/mnt/datalake/book/chapter03/YellowTaxisDeltaPartitioned"
""")

In [ ]:
spark.sql("SELECT COUNT(*) > 0 AS `Partition exists` FROM taxidb.YellowTaxisPartitioned WHERE VendorId = 2 AND RateCodeId = 99")

### Selectively updating Delta partitions with replaceWhere

In [ ]:
spark.sql("SELECT RideId, VendorId, PaymentType FROM taxidb.yellowtaxispartitioned WHERE VendorID = 1 AND RatecodeId = 99 LIMIT 5")

In [ ]:
# Let’s assume that we have a business reason that states that all PaymentTypes for
# VendorId = 1 and RatecodeId = 9 should be 3. We can use the following PySpark
# expression with replaceWhere to achieve that result

# 임의의 조건에 해당하는 데이터 집합을 사전에 생성해 두고 (PaymentType 컬럼은 기 존재하는 컬럼)
from pyspark.sql.functions import *
businessCondition = (
    spark.read.format("delta")
    .load("/mnt/datalake/book/chapter03/YellowTaxisDeltaPartitioned")
    .where((col("VendorId") == 1) & (col("RatecodeId") == 99))
    .withColumn("PaymentType", lit(3))
)

# 해당 데이터프레임 기준으로 존재하는 컬럼에 교체하는 연산을 수행
replaceWhere = (
    businessCondition.write.format("delta")
    .option("replaceWhere", "VendorId = 1 AND RateCodeId = 99")
    .mode("overwrite")
)
replaceWhere.save("/mnt/datalake/book/chapter03/YellowTaxisDeltaPartitioned")

In [ ]:
spark.sql("""
SELECT DISTINCT(PaymentType)
FROM taxidb.yellowtaxispartitioned
WHERE VendorID = 1 AND RatecodeId = 99
""")

### User-Defined Metadata
> Let’s look at the SparkSession configuration first. Assume that we have an INSERT
operation, to which we want to assign a GDPR tag for auditing purposes.

In [ ]:
# Using SparkSession to Set Custom Metadata
spark.sql(f"""
SET spark.databricks.delta.commitInfo.userMetadata=my-custom-metadata= { "GDPR": "INSERT Request 1x965383" }
""")

# The GDPR tag will automatically be applied to the commit info in the transaction log.
spark.sql(f"""
INSERT INTO taxidb.yellowtaxisPartitioned (RideId, VendorId, PickupTime, DropTime,
    PickupLocationId, DropLocationId, CabNumber,
    DriverLicenseNumber, PassengerCount, TripDistance,
    RatecodeId, PaymentType, TotalAmount,
    FareAmount, Extra, MtaTax, TipAmount,
    TollsAmount, ImprovementSurcharge)
VALUES(10000000, 3, '2019-11-01T00:00:00.000Z',
    '2019-11-01T00:02:23.573Z', 65, 71, 'TAC304',
    '453987', 2, 4.5, 1, 1, 20.34, 15.0, 0.5,
    0.4, 2.0, 2.0, 1.1)
""")

In [ ]:
# Using the DataFrameWriter to Set Custom Metadata
customMetadata = df.write.mode("append").format("delta").option("userMetadata", '{"PII": "Confidential XYZ"}')
customMetadata.save("/mnt/datalake/book/chapter03/YellowTaxisDeltaPartitioned")

### Conclusion
> 기본적인 연산 수행 시에, 다양한 방식(SQL DDL/DML, DataFrameWriter API, DeltaTable Builder API 등)으로 수행이 가능하며
테이블 생성 시에도 경로를 직접 명시하여 비관리 테이블로 사용할 수도 있고, 테이블 수준으로 관리 테이블로 활용이 가능하다
파티셔닝을 통해 관리 효율성 및 성능 향상을 기대할 수 있으며, replaceWhere 같은 구문을 통해서 특정 파티션 혹은 조건의 데이터를 효과적으로 업데이트 할 수 있다
또한 이용자 정의 메타데이터를 활용하여 특정 태그를 가진 statements 혹은 operations 목록을 확인할 수 있고, 또한 감사 혹은 규제를 목적으로 활용할 수도 있다 